In [1]:
import socket
import re
import gevent
from gevent import monkey
monkey.patch_all()

Traceback (most recent call last):
  File "src/gevent/_abstract_linkable.py", line 134, in gevent.__abstract_linkable.AbstractLinkable._notify_links
greenlet.error: cannot switch to a different thread
2019-06-14T16:37:44Z (<built-in method switch of greenlet.greenlet object at 0x000002C54DAB5198>, <gevent._event.Event object at 0x000002C5542ABC48>) failed with error



True

D:\ProgramFiles\Anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
def service_client(new_socket):
    request=new_socket.recv(1024).decode("utf-8")
    
    request_lines=request.splitlines()
    
    file_name=""
    print(request_lines[0])
    ret=re.match(r"[^/]+(/[^ ]*)",request_lines[0])
    if ret:
        file_name=ret.group(1)
        if file_name=="/":
            file_name="/index.html"
    try:
        f=open("./html"+file_name,"rb")
    except:
        response="HTTP/1.1 404 NOT FOUND\r\n"
        response+="\r\n"
        response+="file not found"
        new_socket.send(response.encode("utf-8"))
    else:
        html_content=f.read()
        f.close()
        response="HTTP/1.1 200 OK\r\n"
        response+="\r\n"
        new_socket.send(response.encode("utf-8"))
        new_socket.send(html_content)
    new_socket.close()

The history saving thread hit an unexpected error (This operation would block forever
	Hub: <Hub '' at 0x2c553f9ca58 default pending=0 ref=0 thread_ident=0x39d8>
	Handles:
[HandleState(handle=<cdata 'struct uv_handle_s *' 0x000002C54C3CD8E8>, type=b'check', watcher=<loop at 0x2c54daa6748 default pending=0 ref=0>, ref=0, active=1, closing=0),
 HandleState(handle=<cdata 'struct uv_handle_s *' 0x000002C54C362FC8>, type=b'timer', watcher=<loop at 0x2c54daa6748 default pending=0 ref=0>, ref=0, active=1, closing=0),
 HandleState(handle=<cdata 'struct uv_handle_s *' 0x000002C54C3CD998>, type=b'prepare', watcher=<loop at 0x2c54daa6748 default pending=0 ref=0>, ref=0, active=1, closing=0),
 HandleState(handle=<cdata 'struct uv_handle_s *' 0x000002C54C3CDBA8>, type=b'check', watcher=<loop at 0x2c54daa6748 default pending=0 ref=0>, ref=1, active=0, closing=0)]).History will not be written to the database.


In [ ]:
def main():
    #创建套接字
    tcp_server_socket=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    tcp_server_socket.setsockopt(socket.SOL_SOCKET,socket.SO_REUSEADDR,1)
    #绑定
    tcp_server_socket.bind(("",7890))
    #变为监听套接字
    tcp_server_socket.listen(128)
    
    while True:
        #等待新客户端的链接
        new_socket,client_addr=tcp_server_socket.accept()
        gevent.spawn(service_client,new_socket)
    
    tcp_server_socket.close()

In [ ]:
if __name__=='__main__':
    main()